Adapted from example here: https://colab.research.google.com/drive/1vIjBtePIZwUaHWfjfNHzBjwuXOyU_ugD?usp=sharing

In [1]:
# pip installs
!pip install -q -U peft transformers datasets bitsandbytes trl==0.19.1 accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# load model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig
from google.colab import userdata

# Load the 7b llama 3 model
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_id = "meta-llama/Llama-3.2-1B" # for testing purposes

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    token=userdata.get('HF_TOKEN'))

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# Set it to a new token to correctly attend to EOS tokens.
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

1

In [4]:
# add LoRA adapter

lora_config = LoraConfig(
    r=32,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model.add_adapter(lora_config)

In [5]:
# prepare dataset
from datasets import load_dataset

train_dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split="train")

# ds = pd.DataFrame(ds["train"])
# ds = ds[:100]
# ds.head()

# prompt_template = """
# Below is an instruction that describes a task. Write a response that appropriately completes the request.
# ### Instruction:
# You are an experienced medical professional. Answer the following medial question:
# {}
# ### Response:
# <think>
# {}
# </think>
# {}
# """

# ds['text'] = ds.apply(lambda x: prompt_template.format(x['Question'], x['Complex_CoT'], x['Response']), axis=1)
# ds = Dataset.from_pandas(ds[['text']])
# ds = ds.map(lambda x: tokenizer(x['text'], truncation=True), batched=True)

README.md: 0.00B [00:00, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [6]:
# not sure why???
from huggingface_hub import notebook_login

notebook_login()

In [7]:
model_base = model_id.split("/")[-1]
model_name = f"{model_base}-qlora-medical-r32-limited"
YOUR_HF_USERNAME = "erica-landreth"

output_dir = f"{YOUR_HF_USERNAME}/{model_name}"
output_dir

'erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-r32-limited'

In [8]:
# define training arguments and trainer
from transformers import TrainingArguments

model_base = model_id.split("/")[-1]
model_name = model_name
YOUR_HF_USERNAME = "erica-landreth"

output_dir = f"{YOUR_HF_USERNAME}/{model_name}"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 300
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    # push_to_hub=True,
)

In [9]:
train_dataset[0]

{'Question': 'Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?',
 'Complex_CoT': "Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.\n\nSo, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?\n\nOh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that shouldn't be there.\n\nLet's put this together: if a blood clot from the leg somehow travels to the l

In [10]:
# define trainer

from trl import SFTTrainer

# https://stackoverflow.com/questions/78688141/how-to-choose-dataset-text-field-in-sfttrainer-hugging-face-for-my-llm-model
def formatting_func(example):
    text = (f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n"
            f"{example['Question']}<|eot_id|><|start_header_id|>"
            f"assistant<|end_header_id|>\n\n{example['Response']}<|eot_id|>")
    return text

# def formatting_func_general(example,tokenizer):
#     info = {
#         "system": "You are an experienced medical professional.",
#         "user": example['Question'],
#         "assistant": example['Response']
#     }
#     return info

# def formatting_func(example):
#     text = f"""
#     Below is an instruction that describes a task. Write a response that appropriately completes the request.
#     ### Instruction:
#     You are an experienced medical professional. Answer the following medial question:
#     {example['Question']}
#     ### Response:
#     {'Response'}
#     """
#     return text

# def formatting_func(example):
#     text = f"### USER: {example['Question']}\n### ASSISTANT: {example['Response']}"
#     return text

# trainer = SFTTrainer(
#     model=model,
#     args=training_arguments,
#     train_dataset=train_dataset,
#     packing=True,
#     dataset_text_field="id",
#     tokenizer=tokenizer,
#     max_seq_length=1024,
#     formatting_func=formatting_func,
# )

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    # packing=True,
    # dataset_text_field="id",
    # tokenizer=tokenizer,
    # max_seq_length=1024,
    formatting_func=formatting_func,
)

Applying formatting function to train dataset:   0%|          | 0/19704 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/19704 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/19704 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: erica-landreth (erica-landreth-university-of-california-berkeley) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,1.743100
20,1.506700
30,1.430700
40,1.424600
50,1.413100
60,1.357100
70,1.322800
80,1.343600
90,1.306300
100,1.274500


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=300, training_loss=1.299489091237386, metrics={'train_runtime': 4521.8189, 'train_samples_per_second': 1.062, 'train_steps_per_second': 0.066, 'total_flos': 6.413428803767501e+16, 'train_loss': 1.299489091237386})

In [ ]:
# huggingface login
from huggingface_hub import notebook_login

notebook_login()

In [12]:
model.push_to_hub(model_name,token=userdata.get('notebook2'))

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...pnorl8jnw/adapter_model.safetensors:   0%|          | 30.5kB /  109MB            

CommitInfo(commit_url='https://huggingface.co/erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-r32-limited/commit/6b8a47ab6d902e46ddaae920c6ad12a927553d35', commit_message='Upload LlamaForCausalLM', commit_description='', oid='6b8a47ab6d902e46ddaae920c6ad12a927553d35', pr_url=None, repo_url=RepoUrl('https://huggingface.co/erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-r32-limited', endpoint='https://huggingface.co', repo_type='model', repo_id='erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-r32-limited'), pr_revision=None, pr_num=None)